In [1]:
#IMPORT LIBRARIES
import pandas as pd
import sqlite3

In [2]:
#INSTALLING IPYTHON SQL LIBRARY
!pip install ipython-sql

In [3]:
#CREATE CONNECTIONpip install ipython-sql
cnn=sqlite3.connect("international_debt.db")

In [4]:
df=pd.read_csv("international_debt.csv")

In [6]:
df.to_sql("international_debt_ana",cnn)

2357

In [7]:
#LOAD SQL MODULE TO PYHTON
%load_ext sql

In [8]:
%sql sqlite:///international_debt_ana.db

**1.UNDERSTAND THE DATA FIRST**

In [9]:
%%sql
SELECT * FROM international_debt_ana LIMIT 10

 * sqlite:///international_debt_ana.db
(sqlite3.OperationalError) no such table: international_debt_ana
[SQL: SELECT * FROM international_debt_ana LIMIT 10]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


**2.Finding the number of distinct countries**

From the first ten rows, we can see the amount of debt owed by Afghanistan in the different debt indicators. But we do not know the number of different countries we have on the table. There are repetitions in the country names because a country is most likely to have debt in more than one debt indicator.

Without a count of unique countries, we will not be able to perform our statistical analyses holistically. In this section, we are going to extract the number of unique countries present in the table.

In [10]:
%%sql
SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries
FROM international_debt_ana

 * sqlite:///international_debt_ana.db
(sqlite3.OperationalError) no such table: international_debt_ana
[SQL: SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries
FROM international_debt_ana]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


**3.Finding out the distinct debt indicators**

We can see there are a total of 124 countries present on the table. As we saw in the first section, there is a column called indicator_name that briefly specifies the purpose of taking the debt. Just beside that column, there is another column called indicator_code which symbolizes the category of these debts. Knowing about these various debt indicators will help us to understand the areas in which a country can possibly be indebted to.

In [13]:
%%sql
SELECT DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt_ana
ORDER BY distinct_debt_indicators;

 * sqlite:///international_debt_ana.db
(sqlite3.OperationalError) no such table: international_debt_ana
[SQL: SELECT DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt_ana
ORDER BY distinct_debt_indicators;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


**4. Totaling the amount of debt owed by the countries**

As mentioned earlier, the financial debt of a particular country represents its economic state. But if we were to project this on an overall global scale, how will we approach it?

Let's switch gears from the debt indicators now and find out the total amount of debt (in USD) that is owed by the different countries. This will give us a sense of how the overall economy of the entire world is holding up.

In [19]:
%%sql
SELECT 
    ROUND(SUM(debt)/1000000, 2) AS total_debt
FROM international_debt;

 * sqlite:///international_debt.db
Done.


total_debt
3079734.49


**5. Country with the highest debt**

"Human beings cannot comprehend very large or very small numbers. It would be useful for us to acknowledge that fact." - Daniel Kahneman. That is more than 3 million million USD, an amount which is really hard for us to fathom.

Now that we have the exact total of the amounts of debt owed by several countries, let's now find out the country that owns the highest amount of debt along with the amount. Note that this debt is the sum of different debts owed by a country across several categories. This will help to understand more about the country in terms of its socio-economic scenarios. We can also find out the category in which the country owns its highest debt. But we will leave that for now.

In [20]:
%%sql
SELECT 
    country_name, 
    SUM (debt) AS total_debt
FROM international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1;

 * sqlite:///international_debt.db
Done.


country_name,total_debt
China,285793494734.2


**6. Average amount of debt across indicators**

So, it was China. A more in-depth breakdown of China's debts can be found here.

We now have a brief overview of the dataset and a few of its summary statistics. We already have an idea of the different debt indicators in which the countries owe their debts. We can dig even further to find out on an average how much debt a country owes? This will give us a better sense of the distribution of the amount of debt across different indicators.

In [21]:
%%sql
SELECT 
    indicator_code AS debt_indicator,
    indicator_name,
    AVG (debt) AS average_debt
FROM international_debt
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 10;

 * sqlite:///international_debt.db
Done.


debt_indicator,indicator_name,average_debt
DT.AMT.DLXF.CD,"Principal repayments on external debt, long-term (AMT, current US$)",5904868401.499195
DT.AMT.DPNG.CD,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",5161194333.812659
DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS, current US$)",2152041216.8902435
DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.8598351
DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265
DT.INT.DLXF.CD,"Interest payments on external debt, long-term (INT, current US$)",1644024067.6508067
DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.39823
DT.INT.DPNG.CD,"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",1220410844.4215188
DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064
DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.6536236


**7. The highest amount of principal repayments**

We can see that the indicator DT.AMT.DLXF.CD tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital. More information about this category can be found here.

An interesting observation in the above finding is that there is a huge difference in the amounts of the indicators after the second one. This indicates that the first two indicators might be the most severe categories in which the countries owe their debts.

We can investigate this a bit more so as to find out which country owes the highest amount of debt in the category of long term debts (DT.AMT.DLXF.CD). Since not all the countries suffer from the same kind of economic disturbances, this finding will allow us to understand that particular country's economic condition a bit more specifically.

In [22]:
%%sql
SELECT 
    country_name, 
    indicator_name
FROM international_debt
WHERE debt= (SELECT 
                 MAX(debt)
             FROM international_debt
             WHERE indicator_code ='DT.AMT.DLXF.CD');

 * sqlite:///international_debt.db
Done.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"


**8. The most common debt indicator**

China has the highest amount of debt in the long-term debt (DT.AMT.DLXF.CD) category. This is verified by The World Bank. It is often a good idea to verify our analyses like this since it validates that our investigations are correct.

We saw that long-term debt is the topmost category when it comes to the average amount of debt. But is it the most common indicator in which the countries owe their debt? Let's find that out.

In [23]:
%%sql
SELECT 
    indicator_code,
    COUNT (indicator_code) AS indicator_count
FROM international_debt
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC
LIMIT 20;

 * sqlite:///international_debt.db
Done.


indicator_code,indicator_count
DT.INT.OFFT.CD,124
DT.INT.MLAT.CD,124
DT.INT.DLXF.CD,124
DT.AMT.OFFT.CD,124
DT.AMT.MLAT.CD,124
DT.AMT.DLXF.CD,124
DT.DIS.DLXF.CD,123
DT.INT.BLAT.CD,122
DT.DIS.OFFT.CD,122
DT.AMT.BLAT.CD,122


**9. Other viable debt issues and conclusion**

There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. The indicator DT.AMT.DLXF.CD is also there in the list. So, this gives us a clue that all these countries are suffering from a common economic issue. But that is not the end of the story, a part of the story rather.

Let's change tracks from debt_indicators now and focus on the amount of debt again. Let's find out the maximum amount of debt across the indicators along with the respective country names. With this, we will be in a position to identify the other plausible economic issues a country might be going through. By the end of this section, we will have found out the debt indicators in which a country owes its highest debt.

In this notebook, we took a look at debt owed by countries across the globe. We extracted a few summary statistics from the data and unraveled some interesting facts and figures. We also validated our findings to make sure the investigations are correct.

In [24]:
%%sql
SELECT 
    country_name, 
    indicator_code, 
    MAX (debt) AS maximum_debt
FROM international_debt
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 10;

 * sqlite:///international_debt.db
Done.


country_name,indicator_code,maximum_debt
China,DT.AMT.DLXF.CD,96218620835.7
Brazil,DT.AMT.DLXF.CD,90041840304.1
China,DT.AMT.DPNG.CD,72392986213.8
Russian Federation,DT.AMT.DLXF.CD,66589761833.5
Turkey,DT.AMT.DLXF.CD,51555031005.8
South Asia,DT.AMT.DLXF.CD,48756295898.2
Brazil,DT.AMT.PRVT.CD,43598697498.6
Russian Federation,DT.AMT.DPNG.CD,42800154974.9
Brazil,DT.AMT.DPNG.CD,41831444053.3
Least developed countries: UN classification,DT.DIS.DLXF.CD,40160766261.6
